# Counts for Megacatalogue

This notebook contains the counting and assessment of the megacatalogue

The "Getting the Counts" portion will eventually be placed in the Megacatalogue notebook but for easier access can be found here.

The "CDLI Comparison" portion will be part of a new notebook called "CDLI and ORACC Synthesis" and/or replace the Metacatalogue notebook.

# Mount Google Drive folder + imports + steps

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#any necessary imports
import pandas as pd
import zipfile
from zipfile import ZipFile
import json
import requests
from tqdm import tqdm
import os
import errno
import re
import random
import numpy as np
import sys
import copy
import networkx as nx
from pathlib import Path

#Set folder for remote drive
#folder = '/content/drive/My Drive/FactGrid Cuneiform (AWCA)/people/Melinee'
folder = '/content/drive/MyDrive/Melinee/'

#importing utils for the method which downloads the current text json files
os.chdir(folder + 'network/utils/')
from utils import oracc_download

# This is a user defined module that searches through the texts to find the entities in the text that
# are people and places, to be imported as nodes into the network
os.chdir(folder + 'network/')
import rank_parser4 as rp

# Getting the Counts

Import the megacatalogue csv and preview

In [4]:
path = '/content/drive/MyDrive/Melinee/ORACC_DFS/megacatalogue.csv'
megacatalogue = pd.read_csv(path, low_memory=False, index_col=False)

In [5]:
megacatalogue = megacatalogue.drop(columns=['Unnamed: 0'])
megacatalogue.head(3)

,id_text,langs,project,id_text.1,primary_publication,provenience,pleiades_id,pleiades_coord,excavation_no,archive,...,Q_places,parallels,new_q,subseries,subseries_section,description,oracc_id,sec1,chap,sec2
0,P522592,0x08000000,tilbarsip,P522592,Til-Barsip 01,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 01,001 - Hanni Archive (House C1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P522593,0x08000000,tilbarsip,P522593,Til-Barsip 02,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 02,001 - Hanni Archive (House C1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P522594,0x08000000,tilbarsip,P522594,Til-Barsip 03,Tell Ahmar (Til Barsip),658410.0,"[38.1191944, 36.6749623]",T 03,001 - Hanni Archive (House C1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Getting a count for each header, i.e. how many cells have data in them under each header field

In [6]:
counts = megacatalogue.count(axis = 'index')

In [7]:
summary = pd.DataFrame({'count': counts})

In [8]:
summary

,count
id_text,171145
langs,137029
project,171145
id_text.1,161728
primary_publication,99319
...,...
description,0
oracc_id,12
sec1,0
chap,0


In [12]:
summary[summary['count'] == 0].index

Index(['stt_no', 'funder', 'language_remarks', 'principal', 'record_id',
       'repository', 'bibliography__id_biblio', 'year_name_eponym',
       'CDLI_problems', 'has-score', 'reference', 'Non_Sign_List_Series',
       'series_2', 'cdli_collation', 'tablet_number_2', 'tradition', 'corpus',
       'attested', 'composition_designation', 'object_ref', 'new_subgenre',
       'primary_publication2', 'catchline', 'colophon_describing_source',
       'colophon_disclosing_author', 'P_number_problems', 'royal_colophon',
       'series_tablet_no', 'textual_colophon', 'Additional_P_numbers',
       'cdli_id_numbers', 'composite_id', 'elevation', 'Q_designation',
       'exemplar_number', 'findspots', 'btto_attribution', 'century',
       'outline_sort_name', 'subproject', 'modern_converted_date',
       'copy_period', 'c_name', 'c_item', 'c_both', 'c_num_name', 'c_group',
       'special_features', 'hand', 'obta_no', 'sub_genre',
       'additional_P_numbers', 'ancient_day', 'ancient_month',
 